In [1]:
%cd ~/Dropbox/CanvasTools

import environment
import os
import sys
import configparser

import requests

# Load api token and section numbers
print("Reading credentials from %s" % environment.CREDENTIALS_FILE)
config = configparser.ConfigParser()
config.read( environment.CREDENTIALS_FILE)

TOKEN = config['credentials'].get('TOKEN')
SECTION_930 = config['sections'].getint('930')
SECTION_1100 = config['sections'].getint('1100')
SECTION_1230 = config['sections'].getint('1230')
URL_BASE = config['url'].get('BASE')

/Users/adam/Dropbox/CanvasTools
Reading credentials from /Users/adam/Dropbox/CanvasTools/private/canvas-credentials.ini
Reading credentials from /Users/adam/Dropbox/CanvasTools/private/canvas-credentials.ini


In [3]:
from CanvasTools.UrlTools import make_url
from CanvasTools.RequestTools import send_get_request, make_request_header

# Get list of quizzes

 GET /api/v1/courses/:course_id/quizzes 

In [ ]:
# Get quiz list
def get_all_course_quizzes(course_id):
    """Returns a list of all the assignments for the course
    Uses api: GET /api/v1/courses/:course_id/quizzes
    """
    url = make_url(course_id, 'quizzes')
    return send_get_request(url)


def get_quiz(course_id, quiz_id):
    """Retrieves the specified quiz and returns it as a dictionary"""
    url = make_url(course_id, 'quizzes')
    url = "%s/%s" % (url, quiz_id)
    print(url)
#     return send_get_request(url)

In [ ]:
get_all_course_quizzes(SECTION_1230)

In [ ]:
get_quiz(SECTION_930, 100843)

In [ ]:
get_answers(9789591)

In [ ]:
response.status_code

In [ ]:
response.text

In [4]:
quizzes = [(SECTION_930, 100843),(SECTION_1100, 100849), (SECTION_1230, 100850)]

# reports

The URL to use is /api/v1/courses/{CourseId}/quizzes/{QuizId}/reports. Add a form parameter to the header : quiz_report[report_type]=student_analysis. You will get back a Json object and two of its members are a filename and report id. Use the report id to go to /api/v1/courses/{CourseId}/quizzes/{QuizId}/reports/{ReportId}. This will then return the downloaded csv file.

 

In [7]:
course_id = SECTION_1230
quiz_id = 100850

In [15]:
def download_student_reports(quiz_tuples):
    for course_id, quiz_id in quiz_tuples:
        # Create the student report
        url = make_url(course_id, 'quizzes')
        url = "%s/%s/reports" % (url, quiz_id)
        dt = {'quiz_report': {'report_type': 'student_analysis'}, 'include': 'file'}
        print(url)
        response1 = requests.post(url, headers=make_request_header(), json=dt)
        r1 = response1.json()
        # The id of the newly created report
        report_id = r1['id']

        # retrieve the report file information
        url = "%s/%s" % (url, report_id)
        dt = { 'include': 'file'}
        print(url)
        response2 = requests.get(url, headers=make_request_header(), json=dt)
        r2 = response2.json()

        # Link to the resource 
        file_url = r2['file']['url']
        file_name =  "%s %s" % (course_id, r2['file']['display_name'])

        # download file
        print(file_url)
        file_response = requests.get(file_url, headers=make_request_header())

        # write to temp
        destination = "%s/%s" % (environment.TEMP_DATA_PATH, file_name)
        print("Writing to %s" % destination)
        open(destination, 'wb').write(file_response.content)

In [16]:
download_student_reports(quizzes)

https://canvas.csun.edu/api/v1/courses/41179/quizzes/100843/reports
https://canvas.csun.edu/api/v1/courses/41179/quizzes/100843/reports/40084
https://canvas.csun.edu/files/4676929/download?download_frd=1&verifier=z1ERTI1raBBIhIzinTkVp9mpPQx2zihJDXb3cY9S
Writing to /Users/adam/Dropbox/CanvasTools/temp/41179 Persuasive advertising, autonomy, and the creation of desire Quiz Student Analysis Report.csv
https://canvas.csun.edu/api/v1/courses/41180/quizzes/100849/reports
https://canvas.csun.edu/api/v1/courses/41180/quizzes/100849/reports/40141
https://canvas.csun.edu/files/4677319/download?download_frd=1&verifier=RQtHNYuwOPYhpjknh2prf9ZNnpDq4wCeRwDyqyLQ
Writing to /Users/adam/Dropbox/CanvasTools/temp/41180 Persuasive advertising, autonomy, and the creation of desire Quiz Student Analysis Report.csv
https://canvas.csun.edu/api/v1/courses/41181/quizzes/100850/reports
https://canvas.csun.edu/api/v1/courses/41181/quizzes/100850/reports/40142
https://canvas.csun.edu/files/4677347/download?downloa

In [9]:
def download_item_reports(quiz_tuples):
    for course_id, quiz_id in quiz_tuples:
        # Create the item report
        url = make_url(course_id, 'quizzes')
        url = "%s/%s/reports" % (url, quiz_id)
        dt = {'quiz_report': {'report_type': 'item_analysis'}, 'include': 'file'}
        print(url)
        response1 = requests.post(url, headers=make_request_header(), json=dt)
        r1 = response1.json()

        # The id of the newly created report
        report_id = r1['id']

        # retrieve the report file information
        url = "%s/%s" % (url, report_id)
        dt = { 'include': 'file'}
        print(url)
        response2 = requests.get(url, headers=make_request_header(), json=dt)
        r2 = response2.json()

        r2

        # Link to the resource 
        file_url = r2['file']['url']
        file_name =  "%s %s" % (course_id, r2['file']['display_name'])

        # download file
        print(file_url)
        file_response = requests.get(file_url, headers=make_request_header())

        # write to temp
        destination = "%s/%s" % (environment.TEMP_DATA_PATH, file_name)
        print("Writing to %s" % destination)
        open(destination, 'wb').write(file_response.content)

https://canvas.csun.edu/api/v1/courses/41181/quizzes/100850/reports/40143


In [6]:
download_item_reports(quizzes)

https://canvas.csun.edu/api/v1/courses/41179/quizzes/100843/reports
https://canvas.csun.edu/api/v1/courses/41179/quizzes/100843/reports/40085


KeyError: 'file'

# Get and store data about quizzes and questions

## Quiz questions

GET /api/v1/courses/:course_id/quizzes/:quiz_id/questions 

In [18]:
course_id = SECTION_930
quiz_id = 100843

url = make_url(course_id, 'quizzes')
url = "%s/%s/questions" % (url, quiz_id)
print(url)

https://canvas.csun.edu/api/v1/courses/41179/quizzes/100843/questions


In [19]:
response = requests.get(url, headers=make_request_header())
r = response.json()
r


[{'id': 1785087,
  'quiz_id': 100843,
  'quiz_group_id': None,
  'assessment_question_id': 5158108,
  'position': None,
  'question_name': 'Question',
  'question_type': 'essay_question',
  'question_text': "<p>What is an example of persuasive advertising?</p>\n<p>(Don't use Crisp's examples)</p>",
  'points_possible': 1.0,
  'correct_comments': '',
  'incorrect_comments': '',
  'neutral_comments': '',
  'correct_comments_html': '',
  'incorrect_comments_html': '',
  'neutral_comments_html': '',
  'answers': [],
  'variables': None,
  'formulas': None,
  'answer_tolerance': None,
  'formula_decimal_places': None,
  'matches': None,
  'matching_answer_incorrect_matches': None},
 {'id': 1785088,
  'quiz_id': 100843,
  'quiz_group_id': None,
  'assessment_question_id': 5158109,
  'position': None,
  'question_name': 'Question',
  'question_type': 'essay_question',
  'question_text': "<p>What is an example of informational advertising?</p>\n<p>(Don't use Crisp's examples)</p>",
  'points_p

In [ ]:
response.content

# submissions

In [ ]:
# get all quiz submissions
# /api/v1/courses/:course_id/quizzes/:quiz_id/submissions 

In [ ]:
def get_quiz_submissions(course_id, quiz_id):
    url = make_url(course_id, 'quizzes')
    url = "%s/%s/submissions" % (url, quiz_id)
    print(url)
    return send_get_request(url)

In [ ]:
get_quiz_submissions(SECTION_930, 100843)

In [ ]:
 GET /api/v1/quiz_submissions/:quiz_submission_id/questions 

In [ ]:
def get_answers(submission_id):
    url = "https://canvas.csun.edu/api/v1/quiz_submissions/%s/questions" % (submission_id)
    return send_get_request(url)


In [ ]:
submission_id = 1164462
course_id = SECTION_930
quiz_id = 100843

url = make_url(course_id, 'quizzes')
url = "%s/%s/submissions/%s" % (url, quiz_id, submission_id )
print(url)

head = {'Authorization': 'Bearer {}'.format(environment.TOKEN),
       'include': 'quiz_question'
       }
response = requests.get(url, headers=head)


In [ ]:
submission_id = 9789593
# url = "https://canvas.csun.edu/api/v1/quiz_submissions/%s/questions" % (submission_id)
url = "https://canvas.csun.edu/api/v1/courses/41179/quiz_submissions/%s/questions"  % (submission_id)
# url = 'https://canvas.csun.edu/courses/41179/quizzes/100843/history?quiz_submission_id=1158384&version=1'
head = {'Authorization': 'Bearer {}'.format(environment.TOKEN)}
response = requests.get(url, headers=head)


In [ ]:
response.json()